# Signal Spaces
 
```{epigraph}
Programming starts with types and functions. You probably have some preconceptions
about what types and functions are: get rid of them! They will cloud your mind. 
 
--Bartosz Milewski
```

## The model

Signal processing starts with a time series, we apply algorithms on the time series to analyze it. To systemetically organize such processing procedure, we need to model the process.

The signal spaces we are considering is finite discrete spaces whereas the time domain is the  set of natural numbers. The intepretation of this finite set of time axis is not within our model. In this setting, the signal spaces we is simply $\mathbb C^N$, where $N$ is a natural number corresponding to the length of the signal.

We have two classes of time series analysis algorithms, the global analysis and localized analysis. The algebraic data structure (ADS) for the signal is just a list of numbers, we name it *0-sig* with symbol $\mathfrak{F}_0$. 

For localized analysis we have two cases, the block transforms and smooth block transforms.

A block transform basically cut the original signal into serveral sub-signals with the same length and uses the global algorithm performing on each sub-signal. This leads to the notion of *1-sig* which is list of list of numbers or list of 0-sig, the symbol $\mathfrak{F}_1$ is used for this category.

A smooth version of block transform requires extensions on two ends of each sub-signal, we use *3-sig* to denote its ADS, which is a list of triple 0-sig, corresponding to the left extension, the sub-signal itself and the the right extension repectively. Symbolically, this category is identified as $\mathfrak{F}_3$.

For global analysis, we view it as a special case of 1-sig, which contains only one block. This block's ADS is 0-sig representing the original signal.

To summerize, the following is our model:
1. Time domain is digitalize into natural number.
2. $\mathbb C^N$ is the Signal spaces.
3. Three algebraic data structures 0-sig, 1-sig and 3-sig corresponding to original signal, block transforms (global transforms) and smooth block transform repectively. We view these ADS's as container or category, containing blocks of signal with particular structure. A given analysis (or algoritm) is performed on each block.
4. We further require $N=2^L, L,N\in\mathbb N$, preprocessing and time domain intepretation are outside the model.

### The mathematical representation
In mathematical terms, we have defined three underlying signal categories: $\mathfrak{F}_0=\mathbb{C}^N$, $\mathfrak{F}_1=\{\mathbb{C}^M\}_{k=1}^K$ and $\mathfrak{F_3}=\{(C^{M_e},C^M,C^{M_e})\}_{k=1}^K$. Verbally, we name these three categories 0-sig, 1-sig and 3-sig respectively. Each block (for 3-sig, a block is a triple 0-sig) is an object in the corresponding category. There are two distinguashed classes of mappings: mappings in the category level and mappings in the object level. Mappings in the category level are called *morphisms*. There are three classes of morphisms we are interested in:
1. $f:\mathfrak{F}_i\longrightarrow\mathfrak{F}_i,\ \textrm{for}\ i=0,1,3$, this class consists of morphisms that are map from $\mathfrak{F}_i$ to itself. This kind of morphisms is *endomorphism*.
2. $f:\mathfrak{F}_i\longrightarrow\mathfrak{F}_j,\ \textrm{for}\ i,j=0,1,3,i\neq j$, these are the morphisms that go from one of the three signal categories to another one within the three.
3. $f:\mathfrak{F}_i\longrightarrow\mathfrak{F},\ \textrm{for}\ i=0,1,3$, morphisms from within this three categries go outside of the three.

Mappings in object level are called *local  analysis* or *atomic algorithms* (or just simply *algorithms*) in our case, in general they are called *arrows*. These are the functions acting on a particular object in a category and mapping to another object (might be in another category). they are not acting on the categories. In our case, an object is a blocked of original input signal. To be more specific, *algorithms* are the functions on $\mathbb{C}^M$, or symbolically, $\lambda:\mathbb{C}^M\longrightarrow\mathbb{C}^M$ for a given algorithm $\lambda$.

```{note}
One might argue that the three categories we defined is in fact one category with three objects, $\cdot$, $[\cdot]$ and $[[[\cdot],[\cdot],[\cdot]]]$. This is a zoomed out version of our model, it is a different model. We want more detail structures so that we can operate on the `Sig_0` level, which is viewed as the objects in `Sig_1`. But this point of view is fruitful as it points out all three categories share the same outer structure, namely the $[]$.
```

### Implementation
#### Data type
In program, here in haskell, we use different data type representing the above concepts. The three categories are corresponding to: $[\cdot]$, $[[\cdot]]$, $[([\cdot],[\cdot],[\cdot])]$, where
1. $\cdot$ : primitive type, e.g number and string.
2. $[]$: container, e.g. list.

Therefore, for the three categories, we have two classes of types: primitive types and the container.

#### Algorithms and morphisms
The development of a given algorithm for analyzing signals are composed of two phases: the atomic level implementation and the morphism level implementation. An atomic algorithm is acting on an object and give rise to another object may or may not  be the same type. The morphism level implementation orchestrate the atomic alogrithm for the entire category. The implementations of atomic algorithms varies from algorithm to algorithm, while there are only few patterns in the morphism level. One pattern used the most often, for example, is the process of visiting each object in the category and apply the atomic algorithm on it, the resulting set of objects are in the same container structure. These morphism level patterns are called behaviors.

Containers are structures holding objects, we attach to them with behaviors, and turn them into contexts. Each context has one or few morphism level patterns. Implementation of the morphism level turns to construct procedure that utilize context to achieve the need for a given algorithm. So the needs may varies from algorithm to algorithm, but the patters are common to serve as the basic tools to implment each particular need.

Let's first look at a very simple example in `0-Sig`, take the atomic algorithm $\lambda$ to be the `abs` function that take a number and return its absolute value. morphism $f$ maps `0-Sig` to `0-Sig` by applying `abs` to each object in `0-Sig` and assemble the result in `0-Sig` structure as the return. Denoting this morphism as `fmap`, in haskell, `fmap` is defined by:
```haskell
fmap::(a->a)->([a]->[b])
```
here `(a->a)` is a function in object level, while `([a]->[b])` is the map in the category level. In a way, through `fmap`, we promoted `abs` that is in the object level into a morphism in the category level. The implementation of `([a]->[b])` is quite simple, go through each object in `[]`, applying `abs` in each object, collecting all the result, assembe them in `[]` as the return of the morphism.

 The technique that promotes a given atomic algorithm $\alpha$ to morphism $f$, is call lifting. A good frame work for lifting allows us focusing more on developing functions on objects of the categories, which is usually simplyfying the development process. In addition, this also makes it possible to leverage the usage of the existing algorithms has been developed.

Systematically constructing these patterns (contexts) seems like a daunting task. Fortunately, the lifting schemes are quite mature, our efforts mostly are not developing them, rather to learn how to use them effectively.

## Signal categories and localization
As described earler, we categorize our signal spaces into three categories: 0-sig, 1-sig, 3-sig, corresponding to types `[a]`, `[[a]]` and `[[TripleBlocks a]]`, where `a` is a type of `SigType`. We use type synonym `Sig_0`, `Sig_1` and `Sig_3` repectively. They all have  build-in functor, Applicative, Monad mong other many behaviors as we use list as container. `SigType` can be set initially to some numerical type through type synonym.

In [1]:
import Data.Bits

ispower2::Int->Bool
ispower2 n
    | n<=0 = False
    | otherwise = (n .&. (n-1))==0

### 0-sig

In [2]:
:set -XConstraintKinds
type SigType a = (Num a, Fractional a, Show a, Ord a)

What we want is to restrict `Sig_0` to `SigType`,
```haskell
type Sig_0 a = SigType a => [a]
```
This works for ghc > 9.0.2, but not working for ghc 8.10.7 and ghc 9.0.2 we have tested.

In [3]:
:set -XExplicitForAll
type Sig_0 = []

In [4]:
:k Sig_0

Sig_0 :: * -> *

### 1-sig
We use type synonym here. The other method is to use `newtype`. The advantage of using `newtype` is making the code more regulated as one needs to use the type name to construct the `Sig_1` type object  (the same goes to the other types). The price we pay for such regulation is the need to derive almost all useful type classes like `Functor`, `Show`, etc. Worst of all, there are quite a few `prefix` and `infix` operators, such as `take`, `(++)`, we need to reimplement. Although it is not a lot of work to redo all of these, for the moment, we stay with synonym.

`Sig_0`, `Sig_1` and `Sig_3` enforce the underline data type to be `SigType`, but this enforcement is not restricted, it works only if we code within the space of `Sig_0`, `Sig_1` and `Sig_3`. In many cases one can code with `[]`, as the `Sig` system is just the synonym of `[]`. This is the big drawback for using synonym.

In [5]:
type Sig_1 a = [Sig_0 a]

`blocks` is a morphism (category level function) that maps 0-sig to 1-sig with a given size of each element of 1-sig, so the definition is `blocks::n->Sig_0 a->Sig_1 a`. We require the size of each element must be power of 2. The same requirement goes to 0-sig.

In [6]:
blocks::forall a.SigType a=>Int->Sig_0 a->Sig_1 a
blocks n x
    | not $ ispower2 n = []
    | n<=0 = [x]
blocks _ [] = []
blocks n xs = take n xs:blocks n (drop n xs)

In [7]:
blocks 2 [0..7]

[[0.0,1.0],[2.0,3.0],[4.0,5.0],[6.0,7.0]]

In [8]:
fmap sum $ take 5 $ blocks 2 [0..]

[1.0,5.0,9.0,13.0,17.0]

### 3-sig

In [9]:
data TripleBlocks a = TripleBlocks{left::Sig_0 a,center::Sig_0 a,right::Sig_0 a}
type Sig_3 a = [TripleBlocks a]

In [10]:
instance forall a.SigType a => Show (TripleBlocks a) where
    show TripleBlocks{left=a,center=b,right=c} = "[" ++ show a ++ "," ++ show b ++ "," ++ show c ++ "]"++"\n"

In [11]:
:t left

left :: forall {a}. TripleBlocks a -> Sig_0 a

In [12]:
foo=TripleBlocks{left=[1,2],center=[3,4],right=[5,6]}
print foo

[[1.0,2.0],[3.0,4.0],[5.0,6.0]]

In [13]:
print $ left foo
print $ center foo
print $ right foo

[1,2]

[3,4]

[5,6]

In [14]:
:t left

left :: forall {a}. TripleBlocks a -> Sig_0 a

In [15]:
foo = TripleBlocks{left=[1.0,2.0]}

In [16]:
print $ left foo

[1.0,2.0]

### Padding

In [17]:
data Extensions a = Extensions{extleft::Sig_0 a, extright::Sig_0 a}

instance SigType a=>Show (Extensions a) where
    show Extensions{extleft=l,extright=r} = "["++ show l ++ "," ++ show r ++ "]" ++ "\n"

In [18]:
type GetExtensions a = Int->Sig_0 a->Extensions a

In [19]:
zeroPadding'::SigType a=>Int->Sig_0 a->TripleBlocks a
zeroPadding' n a = TripleBlocks{left=replicate n 0, center=a, right=replicate n 0}

In [20]:
zeroPadding::SigType a=>GetExtensions a
zeroPadding n a = Extensions{extleft=replicate n 0, extright=replicate n 0}

In [21]:
symetricPadding::SigType a=>GetExtensions a
symetricPadding n a = Extensions{extleft=lr,extright=rr} where
    lr = reverse $ take n a
    rr = take n $ reverse a

In [22]:
periodicPadding::SigType a=>GetExtensions a
periodicPadding n a = Extensions{extleft=lr,extright=rr} where
    lr = reverse $ take n  $ reverse a
    rr = take n a

#### Implementation using pattern matching

In [23]:
data PaddingMethod=ZERO|PERIODIC|SYMETRIC deriving(Eq)

In [24]:
pad'::SigType a=>PaddingMethod->Int->Sig_0 a->Extensions a
pad' ZERO = zeroPadding
pad' PERIODIC = periodicPadding
pad' SYMETRIC = symetricPadding

In [25]:
a=[1..8]
fmap (\x -> pad' x 4 a) [ZERO,PERIODIC,SYMETRIC]

[[[0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0]]
,[[5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0]]
,[[4.0,3.0,2.0,1.0],[8.0,7.0,6.0,5.0]]
]

### Interface implementation

In [26]:
class Padding f where
    pad::(SigType a)=>f->(Int->Sig_0 a-> Extensions a)

In [27]:
instance Padding PaddingMethod where
    pad ZERO = zeroPadding
    pad PERIODIC = periodicPadding
    pad SYMETRIC = symetricPadding

In [28]:
a=[1..8]
fmap (\x -> pad x 4 a) [ZERO,PERIODIC,SYMETRIC]

[[[0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0]]
,[[5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0]]
,[[4.0,3.0,2.0,1.0],[8.0,7.0,6.0,5.0]]
]

#### `oblocks` --- A `1-Sig` to `3-Sig` morphism
```haskell
oblocks a::Int->Sig_1 a->Sig_3 a
```

In [29]:
oblocksInner::Int->Sig_1 a->Sig_3 a
oblocksInner n (x:xs)
    | n > length x = error "illegal length"
    | length (take 3 (x : xs))  < 3 = [] -- `take 3` intended to make it work for infinite sequence
    | otherwise = TripleBlocks{left=drop (length x-n) $ head w,center=w!!1,right=take n $ last w}:oblocksInner n xs
    where
        w=take 3 (x:xs)

In [30]:
oblocks::SigType a=>PaddingMethod->Int->Sig_1 a->Sig_3 a
oblocks pm n xs
    | pm == PERIODIC = oblocksInner n (last xs : xs ++ [head xs])
    | pm == ZERO = oblocksInner n (replicate n 0:xs ++ [replicate n 0])
    | pm == SYMETRIC = oblocksInner n (reverse (head xs):xs++[reverse $ last xs])
    | otherwise = error "unkown method"

In [31]:
a=[1..8]
b=blocks 4 a
fmap (\x -> oblocks x 2 b) [PERIODIC,ZERO,SYMETRIC]

[[[[7.0,8.0],[1.0,2.0,3.0,4.0],[5.0,6.0]]
,[[3.0,4.0],[5.0,6.0,7.0,8.0],[1.0,2.0]]
],[[[0.0,0.0],[1.0,2.0,3.0,4.0],[5.0,6.0]]
,[[3.0,4.0],[5.0,6.0,7.0,8.0],[0.0,0.0]]
],[[[2.0,1.0],[1.0,2.0,3.0,4.0],[5.0,6.0]]
,[[3.0,4.0],[5.0,6.0,7.0,8.0],[8.0,7.0]]
]]

In [129]:
generation a b= [[a,b]]
[1,2]>>=generation 1

[[1,1],[1,2]]

In [ ]:
fmap2::(Functor f)=>(a->b->c)->f a->f b->f c
fmap2 

### Sig
The type class `Sig` federalizes the behavior of `fmap` by providing a new function `transform`, but it is too restrictive. What we intend to do here to restrict `transform` so that function `([a]->[b])` is permitted but function `([a]->b)` (e.g. `sum`) is not allowed, in other words we want to preserve the `Sig_1` structure. We don't know how to do this just yet, but it is the placeholder for future development on this issue.

In [32]:
class Sig s where
    transform::(a->a)->s a->s a

In [33]:
instance Sig [] where
    transform=fmap

In [34]:
a = [[1,2],[3,4]]
b = [[5,6],[7,8]]

In [35]:
a++b

[[1,2],[3,4],[5,6],[7,8]]

In [36]:
transform (take 2) $ a++b

[[1,2],[3,4],[5,6],[7,8]]

In [37]:
fmap sum $ a++b

[3,7,11,15]

## Decimation

In [38]:
decimation::Int->[a]->[a]
decimation n x
    | n<2 = x
decimation _ [] = []
decimation n (x:xs) = x:decimation n (drop (n-1) xs)

In [39]:
decimation 2 [1,2,3]

[1,3]

In [40]:
take 5 $ decimation 2 [0..]

[0,2,4,6,8]

In [41]:
decimation 1 [1..5]

[1,2,3,4,5]

## Translation

## Utils
### Linespace

```{note}
Integral literals written using scientific notation will be desugared using fromInteger, whereas any literals which aren’t integral will be desugared using fromRational as usual.
```

In [42]:
linespace' :: SigType a=>a->a->Int->[a]
linespace' _ _ n
    | n<=1 = []
linespace' a1 a2 _
    | a2<=a1 = []
linespace' a1 a2 n = map (\ i -> a1+dt*fromIntegral i) [0..(n-1)]
    where
        dt=(a2-a1)/fromIntegral(n-1)
--linespace a1 a2 n = [let dt=(a2-a1)/(n-1) in a1+i*dt | i<-[0..(n-1)]]

```{warning}
The fromIntegral function is actually unsafe and should be used
with care. The problem is that this function doesn’t check for underflows and
overflows in data. This could potentially lead to corrupting data when doing
incompatible type conversion.
```

In [43]:
linespace :: (SigType a, Integral n)=>a->a->n->Sig_0 a
linespace a1 a2 _
    | a2<=a1 = []
linespace a b n
    | n<=0 = []
    | n==1 = [a, b]
linespace a b n = [let dt=(b-a)/fromIntegral(n-1) in a+dt*fromIntegral i | i<-[0..(n-1)]]

In [44]:
abs <$> linespace (-1) 1 8

[1.0,0.7142857142857143,0.4285714285714286,0.1428571428571429,0.1428571428571428,0.4285714285714284,0.7142857142857142,1.0]

In [45]:
fmap sum $ blocks 2 $ linespace (-1) 1 8

[-1.7142857142857144,-0.5714285714285715,0.5714285714285712,1.7142857142857142]

In [46]:
linespace 1.0 (-1) 1

[]

In [47]:
len::[a]->Int
len [] = 0
len xs = foldr (\ x -> (+) 1) 0 xs

In [48]:
len [0..7]

8

In [49]:
foo=len [0..]

#### Sliding windows --- windowed::Int->[a]->[[a]]

In [50]:
-- windowed on list, returning windows with n elements  each
windowed :: Int->[a]->[[a]]
windowed n x
    | n<=1 = [x]
windowed _ [] = []
windowed n (x:xs)
    | length(take n (x:xs))<n = [] -- `take n` makes sure it works on infinite list
    | otherwise = w:windowed n xs
    where w=take n (x:xs)
-- without guarantee each window has n element, then the following works:
-- windowed n x = Data.List.transpose $ take n $ tails x
-- or
-- windowed n x = map (tak n) $ tails x



In [51]:
take 5 $ windowed 3 [0..]

[[0,1,2],[1,2,3],[2,3,4],[3,4,5],[4,5,6]]

In [52]:
windowed 3 [0..7]

[[0,1,2],[1,2,3],[2,3,4],[3,4,5],[4,5,6],[5,6,7]]

### Binary Tree

In [53]:
data Tree a = EmptyTree | Node a (Tree a) (Tree a) deriving (Show, Read, Eq) 

## Visualization

In [54]:
:set -XExtendedDefaultRules
import Graphics.Matplotlib

: 

In [1]:
import Graphics.Rendering.Chart

: 

In [2]:
import Graphics.Rendering.Chart.Backend.Diagrams

: 

In [3]:
import Diagrams.TwoD.Apollonian

: 